In [2]:
import datetime

In [1]:
MODEL_NAME = "fulltime_draw_outcome"
MODEL_VERSION = 1.0

In [3]:
# UPDATING THE GITHUB OF THE DATASETS
MODEL_TIME = datetime.datetime.now().strftime("%d-%M-%Y_%T")
!git add -A
!git commit -m "updates: {MODEL_NAME}_v{MODEL_VERSION}: bot commit - {MODEL_TIME}"
!git push

[main de923e4] updates: fulltime_draw_outcome_v1.0: bot commit - 30-34-2024_13:34:49
 4 files changed, 4 insertions(+)
 create mode 100644 models/Data Analysis - Data Sheets.xlsx
 create mode 100644 models/fulltime_draw_outcome-v1.0-model.joblib
 create mode 100644 models/fulltime_draw_outcome-v1.0-transformer.joblib
Enumerating objects: 10, done.
Counting objects: 100% (10/10), done.
Delta compression using up to 2 threads
Compressing objects: 100% (7/7), done.
Writing objects: 100% (7/7), 78.09 KiB | 6.51 MiB/s, done.
Total 7 (delta 3), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (3/3), completed with 3 local objects.
To https://github.com/lebyanelm/neural-trained-models.git
   fa2e122..de923e4  main -> main
